In [1]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Update the is_admin column to True (1) for a specific user (e.g., user with id = 1)
cursor.execute("UPDATE user SET is_admin = 1 WHERE id = 1")

# Commit the changes and close the connection
conn.commit()
conn.close()

In [5]:
# Insert sample orders into the "order" table
import sqlite3
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

orders = [
    ('Completed', 2, 99.99, '2025-06-24 21:53:13.712050', 'Credit Card', 'web'),
    ('Cancelled', 2, 150.00, '2025-06-24 21:53:13.712050', 'Credit Card', 'web')
]

#cursor.executemany("""    INSERT INTO "order" (status, user_id, total_amount, timestamp, payment_method, device)    VALUES (?, ?, ?, ?, ?, ?)""", orders)

# Update all 'web' devices to 'Mobile'
cursor.execute("""
    UPDATE "order"
    SET device = 'Mobile'
    WHERE device = 'web'
""")

conn.commit()
conn.close()

In [3]:
import numpy as np
import pandas as pd
# 2. Load and use for predictions
fraud_predictor = FraudPredictor('hybrid_fraud_model.pkl')

# Example: Predict on a sample transaction
sample_transaction = {
    'account_age_days': 30,
    'payment_method': 'Credit Card',
    'device': 'Mobile',
    'category': 'Electronics',
    'amount': 250.0,
    'num_trans_24h': 3,
    'num_failed_24h': 0,
    'no_of_cards_from_ip': 1,
    'transaction_hour': 14
}
sample_transaction['amount_log'] = np.log1p(sample_transaction['amount'])
sample_transaction['hour_sin'] = np.sin(2 * np.pi * sample_transaction['transaction_hour'] / 24)
sample_transaction['hour_cos'] = np.cos(2 * np.pi * sample_transaction['transaction_hour'] / 24)

prediction = fraud_predictor.predict(sample_transaction)
print(prediction)

NameError: name 'FraudPredictor' is not defined

In [4]:
import pandas as pd
import numpy as np
import joblib
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'shap'

In [ ]:


# 0. Define feature lists (adjust as needed for your data)
numerical_features = [
    'account_age_days', 'amount', 'num_trans_24h', 'num_failed_24h', 'no_of_cards_from_ip',
    'transaction_hour', 'amount_log', 'amount_to_avg', 'new_device_flag', 'hour_sin', 'hour_cos'
]
categorical_features = [
    'payment_method', 'device', 'category'
]

# 1. Load and preprocess data
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    
    # Feature engineering
    df['transaction_hour'] = pd.to_datetime(df['TimeStamp']).dt.hour
    df['amount_log'] = np.log1p(df['amount'])
    df['amount_to_avg'] = df['amount'] / df.groupby('User_id')['amount'].transform('mean')
    df['new_device_flag'] = (df['device'] != df.groupby('User_id')['device'].shift(1)).astype(int)
    df['hour_sin'] = np.sin(2 * np.pi * df['transaction_hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['transaction_hour'] / 24)
    
    # Preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ])
    
    # Prepare data
    X = df.drop(['fraud_label', 'fraud_pattern', 'Transaction_id', 'TimeStamp', 
                 'billing_address', 'shipping_address', 'ip_address', 'geo_location'], axis=1)
    y = df['fraud_label']
    
    return X, y, preprocessor

# 2. Train hybrid model
def train_hybrid_model(X, y, preprocessor):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    xgb_model = XGBClassifier(
        scale_pos_weight=45,
        n_estimators=500,
        max_depth=7,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='auc'
    )
    xgb_model.fit(X_train_processed, y_train)
    
    iso_model = IsolationForest(
        n_estimators=500,
        contamination=0.03,
        max_samples=256,
        random_state=42
    )
    iso_model.fit(X_train_processed)
    
    iso_scores = iso_model.decision_function(X_train_processed)
    iso_thresh = np.percentile(iso_scores, 3)
    
    return xgb_model, iso_model, iso_thresh, preprocessor

# 3. Save model to pickle
def save_model(model, path='hybrid_fraud_model_3_outputs.pkl'):
    joblib.dump(model, path)

# 4. Prediction with explainability and custom decision logic
class FraudPredictor:
    def _init_(self, model_path):
        self.model_data = joblib.load(model_path)
        self.xgb_model = self.model_data['xgb_model']
        self.iso_model = self.model_data['iso_model']
        self.preprocessor = self.model_data['preprocessor']
        self.iso_thresh = self.model_data['iso_thresh']
        self.feature_names = self.model_data['feature_names']
        self.xgb_explainer = shap.TreeExplainer(self.xgb_model)
        
    def predict(self, transaction_data):
        processed = self.preprocessor.transform(pd.DataFrame([transaction_data]))
        xgb_prob = self.xgb_model.predict_proba(processed)[0][1]
        iso_score = self.iso_model.decision_function(processed)[0]
        
        # Custom decision logic
        if xgb_prob > 0.8:
            decision = "FRAUD"
        elif (xgb_prob > 0.7) or ((xgb_prob > 0.4) and (iso_score < self.iso_thresh)):
            decision = "NEED TO TAKE FEEDBACK"
        else:
            decision = "GENUINE"
        
        # SHAP explanation
        shap_values = self.xgb_explainer.shap_values(processed)[0]
        indicators = []
        total_impact = sum(np.abs(shap_values))
        for i, val in enumerate(shap_values):
            if val > 0:
                impact_percent = (abs(val) / total_impact) * 100
                indicators.append({
                    'feature': self.feature_names[i],
                    'value': processed[0][i],
                    'impact_percent': round(impact_percent, 1)
                })
        indicators = sorted(indicators, key=lambda x: x['impact_percent'], reverse=True)[:5]
        
        return {
            'decision': decision,
            'probability': round(float(xgb_prob), 4),
            'anomaly_score': round(float(iso_score), 4),
            'fraud_indicators': indicators,
            'thresholds': {
                'xgb_high': 0.7,
                'xgb_medium': 0.4,
                'iso_threshold': round(float(self.iso_thresh), 4)
            }
        }

# ==================== USAGE ==================== #

# 1. Train and save model
X, y, preprocessor = load_and_preprocess_data('fraud_transactions.csv')
xgb_model, iso_model, iso_thresh, preprocessor = train_hybrid_model(X, y, preprocessor)

# Get feature names for SHAP
feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out())

# Prepare model package
model_package = {
    'xgb_model': xgb_model,
    'iso_model': iso_model,
    'preprocessor': preprocessor,
    'feature_names': feature_names,
    'iso_thresh': iso_thresh
}

# Save model
save_model(model_package)

# 2. Load and use for predictions
fraud_predictor = FraudPredictor('hybrid_fraud_model.pkl')

# Example: Predict on a sample transaction
sample_transaction = {
    'account_age_days': 30,
    'payment_method': 'Credit Card',
    'device': 'Mobile',
    'category': 'Electronics',
    'amount': 250.0,
    'num_trans_24h': 3,
    'num_failed_24h': 0,
    'no_of_cards_from_ip': 1,
    'transaction_hour': 14
}
sample_transaction['amount_log'] = np.log1p(sample_transaction['amount'])
sample_transaction['hour_sin'] = np.sin(2 * np.pi * sample_transaction['transaction_hour'] / 24)
sample_transaction['hour_cos'] = np.cos(2 * np.pi * sample_transaction['transaction_hour'] / 24)

prediction = fraud_predictor.predict(sample_transaction)
print(prediction)


In [6]:
import joblib
import numpy as np
import pandas as pd

# Load the saved model package
model_data = joblib.load('../models/hybrid_fraud_model_3_outputs.pkl')
xgb_model = model_data['xgb_model']
iso_model = model_data['iso_model']
preprocessor = model_data['preprocessor']
feature_names = model_data['feature_names']
iso_thresh = model_data['iso_thresh']

# Example test data (replace with your own as needed)
test_sample = {
    'User_id': 1,  # Add User_id for groupby
    'account_age_days': 15,
    'payment_method': 'Credit Card',
    'device': 'Mobile',
    'category': 'Electronics',
    'amount': 120.0,
    'num_trans_24h': 2,
    'num_failed_24h': 0,
    'no_of_cards_from_ip': 1,
    'transaction_hour': 10
}
test_sample['amount_log'] = np.log1p(test_sample['amount'])
test_sample['hour_sin'] = np.sin(2 * np.pi * test_sample['transaction_hour'] / 24)
test_sample['hour_cos'] = np.cos(2 * np.pi * test_sample['transaction_hour'] / 24)

# Simulate a reference DataFrame for groupby calculations
# In real use, replace this with actual user transaction history
ref_df = pd.DataFrame([
    {'User_id': 1, 'amount': 100.0, 'device': 'Mobile'},
    {'User_id': 1, 'amount': 140.0, 'device': 'Mobile'},
    {'User_id': 1, 'amount': 120.0, 'device': 'Mobile'},
])

# Calculate amount_to_avg for the test sample
user_mean = ref_df[ref_df['User_id'] == test_sample['User_id']]['amount'].mean()
test_sample['amount_to_avg'] = test_sample['amount'] / user_mean if user_mean else 1.0

# Calculate new_device_flag for the test sample
# Compare with last device used by this user
last_device = ref_df[ref_df['User_id'] == test_sample['User_id']]['device'].iloc[-1]
test_sample['new_device_flag'] = int(test_sample['device'] != last_device)

# Preprocess and predict
test_df = pd.DataFrame([test_sample])
test_df = test_df.drop(columns=['User_id'])  # Remove User_id before transform
test_processed = preprocessor.transform(test_df)
xgb_prob = xgb_model.predict_proba(test_processed)[0][1]
iso_score = iso_model.decision_function(test_processed)[0]
is_fraud = (xgb_prob > 0.7) or ((xgb_prob > 0.4) and (iso_score < iso_thresh))

print({
    'is_fraud': bool(is_fraud),
    'probability': float(xgb_prob),
    'anomaly_score': float(iso_score),
    'thresholds': {
        'xgb_high': 0.7,
        'xgb_medium': 0.4,
        'iso_threshold': float(iso_thresh)
    }
})


{'is_fraud': False, 'probability': 0.0001516529591754079, 'anomaly_score': 0.08549162716554715, 'thresholds': {'xgb_high': 0.7, 'xgb_medium': 0.4, 'iso_threshold': 1.2206512393542917e-17}}


In [12]:
import pandas as pd
import numpy as np
import joblib
import shap

# Define the FraudPredictor class (must match how you saved the model)
class FraudPredictor:
    def __init__(self, model_path):
        self.model_data = joblib.load(model_path)
        self.xgb_model = self.model_data['xgb_model']
        self.iso_model = self.model_data['iso_model']
        self.preprocessor = self.model_data['preprocessor']
        self.iso_thresh = self.model_data['iso_thresh']
        self.feature_names = self.model_data['feature_names']
        self.xgb_explainer = shap.TreeExplainer(self.xgb_model)
        
    def predict(self, transaction_data):
        processed = self.preprocessor.transform(pd.DataFrame([transaction_data]))
        xgb_prob = self.xgb_model.predict_proba(processed)[0][1]
        iso_score = self.iso_model.decision_function(processed)[0]
        if xgb_prob > 0.8:
            decision = "FRAUD"
        elif (xgb_prob > 0.7) or ((xgb_prob > 0.4) and (iso_score < self.iso_thresh)):
            decision = "NEED TO TAKE FEEDBACK"
        else:
            decision = "GENUINE"
        shap_values = self.xgb_explainer.shap_values(processed)[0]
        indicators = []
        total_impact = sum(np.abs(shap_values))
        for i, val in enumerate(shap_values):
            if val > 0:
                impact_percent = (abs(val) / total_impact) * 100
                indicators.append({
                    'feature': self.feature_names[i],
                    'value': processed[0][i],
                    'impact_percent': round(impact_percent, 1)
                })
        indicators = sorted(indicators, key=lambda x: x['impact_percent'], reverse=True)[:5]
        return {
            'decision': decision,
            'probability': round(float(xgb_prob), 4),
            'anomaly_score': round(float(iso_score), 4),
            'fraud_indicators': indicators,
            'thresholds': {
                'xgb_high': 0.7,
                'xgb_medium': 0.4,
                'iso_threshold': round(float(self.iso_thresh), 4)
            }
        }

# Load the model
fraud_predictor = FraudPredictor('../models/hybrid_fraud_model_3_outputs.pkl')

# Example: Predict on a sample transaction
sample_transaction = {
    'User_id': 1,  # Add User_id for groupby
    'account_age_days': 30,
    'payment_method': 'Credit Card',
    'device': 'Mobile',
    'category': 'Electronics',
    'amount': 250.0,
    'num_trans_24h': 3,
    'num_failed_24h': 0,
    'no_of_cards_from_ip': 1,
    'transaction_hour': 14
}
sample_transaction['amount_log'] = np.log1p(sample_transaction['amount'])
sample_transaction['hour_sin'] = np.sin(2 * np.pi * sample_transaction['transaction_hour'] / 24)
sample_transaction['hour_cos'] = np.cos(2 * np.pi * sample_transaction['transaction_hour'] / 24)

# Calculate amount_to_avg for the sample transaction
user_mean = ref_df[ref_df['User_id'] == sample_transaction['User_id']]['amount'].mean()
sample_transaction['amount_to_avg'] = sample_transaction['amount'] / user_mean if user_mean else 1.0

# Calculate new_device_flag for the sample transaction
last_device = ref_df[ref_df['User_id'] == sample_transaction['User_id']]['device'].iloc[-1]
sample_transaction['new_device_flag'] = int(sample_transaction['device'] != last_device)

# Remove User_id before prediction if not needed by the model
sample_transaction_for_pred = sample_transaction.copy()
sample_transaction_for_pred.pop('User_id', None)

prediction = fraud_predictor.predict(sample_transaction_for_pred)
print(prediction)


{'decision': 'GENUINE', 'probability': 0.0002, 'anomaly_score': 0.0842, 'fraud_indicators': [{'feature': 'payment_method_Net Banking', 'value': np.float64(0.0), 'impact_percent': np.float32(0.1)}, {'feature': 'category_Clothing', 'value': np.float64(0.0), 'impact_percent': np.float32(0.1)}, {'feature': 'category_Groceries', 'value': np.float64(0.0), 'impact_percent': np.float32(0.1)}, {'feature': 'device_Desktop', 'value': np.float64(0.0), 'impact_percent': np.float32(0.0)}], 'thresholds': {'xgb_high': 0.7, 'xgb_medium': 0.4, 'iso_threshold': 0.0}}
